In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "16767493099498662495", 2, "feature")

In [4]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [5]:
y = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "6683389019738139842", 2, "feature")

## Vecchia funzione

In [6]:
function box(model)
	if isa(model,Matrix)
		return nothing
	elseif isa(model,Lar.Struct)
		listOfModels = Lar.evalStruct(model)
		#dim = checkStruct(listOfModels)
		if listOfModels == []
			return model.box
		else
			theMin,theMax = box(listOfModels[1])
			for theModel in listOfModels[2:end]
				modelMin,modelMax = box(theModel)
				for (k,val) in enumerate(modelMin)
					if val < theMin[k]
						theMin[k]=val
					end
				end
				for (k,val) in enumerate(modelMax)
					if val > theMax[k]
						theMax[k]=val
					end
				end
			end
		end
		return [theMin,theMax]

	elseif (isa(model,Tuple) ||isa(model,Array))&& (length(model)>=2)
		V = model[1]
		theMin = minimum(V, dims=2)
		theMax = maximum(V, dims=2)
	end

	return [theMin,theMax]
end

box (generic function with 1 method)

In [7]:
@btime box(x)

  5.980 μs (53 allocations: 2.72 KiB)


2-element Array{Array{Float64,2},1}:
 [0.0; 0.0]
 [1.0; 1.0]

In [8]:
@btime box(y)

  15.600 μs (118 allocations: 5.55 KiB)


2-element Array{Array{Float64,2},1}:
 [-0.5; -0.5]
 [0.5; 0.5]

In [ ]:
@benchmark box(y)

## Nuove funzioni

In [9]:
function evalBox(listOfModels)
	theMin,theMax = box(listOfModels[1])
	for theModel in listOfModels[2:end]
		modelMin,modelMax= box(theModel)
		for (k,val) in enumerate(modelMin)
			if val < theMin[k]
				theMin[k]=val
			end
		end
		for (k,val) in enumerate(modelMax)
			if val > theMax[k]
				theMax[k]=val
			end
		end
	end
	return theMin,theMax
end

evalBox (generic function with 1 method)

In [10]:
function box(model)
	if isa(model,Matrix)
		return nothing
	elseif isa(model,Lar.Struct)
		listOfModels = Lar.evalStruct(model)
		#dim = checkStruct(listOfModels)
		if listOfModels == []
			return model.box
		else
			theMin,theMax = evalBox(listOfModels)
		end
        
		return [theMin,theMax]

	elseif (isa(model,Tuple) || isa(model,Array)) && (length(model)>=2)
		V = model[1]
		theMin = minimum(V, dims=2)
		theMax = maximum(V, dims=2)
	end
    
	return [theMin,theMax]
end

box (generic function with 1 method)

In [11]:
@btime box(x)

  5.933 μs (53 allocations: 2.72 KiB)


2-element Array{Array{Float64,2},1}:
 [0.0; 0.0]
 [1.0; 1.0]

In [12]:
@btime box(y)

  15.300 μs (116 allocations: 5.48 KiB)


2-element Array{Array{Float64,2},1}:
 [-0.5; -0.5]
 [0.5; 0.5]

In [13]:
@benchmark box(y)

BenchmarkTools.Trial: 
  memory estimate:  5.48 KiB
  allocs estimate:  116
  --------------
  minimum time:     15.200 μs (0.00% GC)
  median time:      17.101 μs (0.00% GC)
  mean time:        22.729 μs (2.78% GC)
  maximum time:     6.387 ms (98.78% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [14]:
@code_typed box(y)

CodeInfo(
1 ──       goto #3 if not false
2 ──       nothing::Nothing
3 ┄─ %3  = Base.getproperty(Main.Lar, :Struct)::Any
│    %4  = (model isa %3)::Bool
└───       goto #12 if not %4
4 ── %6  = Base.getproperty(Main.Lar, :evalStruct)::Any
│    %7  = (%6)(model)::Any
│    %8  = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Any,1}, svec(Any, Int64), 0, :(:ccall), Array{Any,1}, 0, 0))::Array{Any,1}
│    %9  = Main.:(==)::Core.Compiler.Const(==, false)
│    %10 = (isa)(%7, WeakRef)::Bool
└───       goto #6 if not %10
5 ── %12 = π (%7, WeakRef)
│    %13 = invoke %9(%12::WeakRef, %8::Array{Any,1})::Union{Missing, Bool}
└───       goto #9
6 ── %15 = (isa)(%7, Missing)::Bool
└───       goto #8 if not %15
7 ──       goto #9
8 ── %18 = (%7 == %8)::Union{Missing, Bool}
└───       goto #9
9 ┄─ %20 = φ (#5 => %13, #7 => $(QuoteNode(missing)), #8 => %18)::Union{Missing, Bool}
└───       goto #11 if not %20
10 ─ %22 = Base.getfield(model, :box)::Any
└───       return %22
11 ─ %24 = Main.evalBox(

## Modifiche

"""\
evalBox(listOfModels::Array)::Tuple\
Genera e restituisce  il volume di contenimento, ovvero il minimo parallelepipedo parallelo agli assi che li contiene, se l'input è una struttura.\
"""

In [10]:
@inline function evalBox(listOfModels)
	theMin,theMax = box(listOfModels[1])
	for theModel in listOfModels[2:end]
		modelMin,modelMax= box(theModel)
		for (k,val) in enumerate(modelMin)
			if val < theMin[k]
				theMin[k]=val
			end
		end
		for (k,val) in enumerate(modelMax)
			if val > theMax[k]
				theMax[k]=val
			end
		end
	end
	return theMin,theMax
end

evalBox (generic function with 1 method)

In [22]:
@inline function box(model)
	if isa(model,Matrix)
		return nothing
	elseif isa(model,Lar.Struct)
		listOfModels = Lar.evalStruct(model)
		if listOfModels == []
			return model.box
		else
			theMin,theMax = evalBox(listOfModels)
		end
        
		return [theMin,theMax]

	elseif (isa(model,Tuple) || isa(model,Array)) && (length(model)>=2)
		V = model[1]
		theMin = minimum(V, dims=2)
		theMax = maximum(V, dims=2)
	end
    
	return [theMin,theMax]
end

box (generic function with 1 method)

In [23]:
@btime box(x)

  5.883 μs (52 allocations: 2.63 KiB)


Tuple{Array{Float64,2},Array{Float64,2}}

In [17]:
@btime box(y)

  15.300 μs (115 allocations: 5.39 KiB)


([-0.5; -0.5], [0.5; 0.5])

In [ ]:
@benchmark box(y)